<a href="https://colab.research.google.com/github/Seng-Pan/ml_project/blob/main/Food_Assistance_Project_main.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# FOOD ASSISTANCE


# Main tasks
- Calories Calculator
- Cook Helper (Recommend Recipes)
- Dietary Filter
- Leftover Remixer
- +allergic avoider

# Flow of the application
- Encode the query image and text with multimodal embedding
- Use Embedded query (image + text) to query information in the RAG
- Retrive the data (image + text) from the RAG, Augmented the prompt
- Prompt the multimodal llm to get responds

# Flow of development
- Fine-tune the multimodal embedding
  - load dataset
    - mrdbourke/food_vision_199_classes (food image, food label)
    - Multimodal-Fatima/Food101_train (food image, food label, full description)
    - Scuccorese/food-ingredients-dataset (food image, ingredients, category)
    - mbien/recipe_nlg (label, recipe)
    - pinkieseb/nutrition_dataset (food image, nutritions**)
    - GPTasty/nutrition-positive-pairs (contrastive learning, positive pair nutrition)
    - Hieu-Pham/kaggle_food_recipes (label, ingredient, instruction)
- Implement RAG vector database system with fine-tuned multimodal embedding + faiss
  - load dataset
    - juliensimon/food102 (food image, food label) (food101 + extra class generated with stable diffusion)
- Construct prompt for llm
- Connect everything together   

In [ ]:
!python --version

Python 3.11.12


In [ ]:
# install all dependencies
!pip install torch
!pip install torchvision

!pip install evaluate
!pip install accelerate

!pip install transformers
# !pip install datasets
!pip install gradio
!pip install huggingface-hub

!pip install faiss-cpu

!pip install sentence-transformers

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 37.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 46.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 34.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 13.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 6.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 6.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 48.4 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstalling nvidia-nvjitlink-cu12-12.5.82:
      Successfully uninstalled nvidia-nvjitlin

In [ ]:
!pip install streamlit
!pip install streamlit-tags

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.3/44.3 kB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.9/9.9 MB 30.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 50.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.1/79.1 kB 5.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.7/2.7 MB 24.2 MB/s eta 0:00:00


In [ ]:
!pip install -U bitsandbytes
!pip install kagglehub

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.1/76.1 MB 8.0 MB/s eta 0:00:00


In [ ]:
!pip install faiss-gpu


In [ ]:
# load all dependencies
from transformers import CLIPModel, CLIPProcessor, pipeline, TextStreamer, BitsAndBytesConfig, LlavaNextForConditionalGeneration, AutoProcessor
from sentence_transformers import SentenceTransformer, CrossEncoder
# from datasets import load_dataset

# faiss or chromaDB

from PIL import Image
from IPython.display import display, HTML

import faiss
import numpy as np
import pandas as pd

import torch

import gradio

import requests
from pathlib import Path
import os
import sys
import json
import math
import re

import time
from tqdm import tqdm
from tqdm import notebook

## Utility Function

In [ ]:
# utility function
# move to python file later
def get_device() -> str:
  if torch.cuda.is_available():
    return "cuda"
  elif torch.backends.mps.is_available():
    return "mps"
  else:
    return "cpu"

In [ ]:
def load_embedded_data(file_name: str) -> np.ndarray:
  return np.load(NLP_DATABASE / f"{file_name}.npy" )

In [ ]:
def load_faiss_index(index_name: str):
  return faiss.read_index(str(NLP_DATABASE / f"{index_name}.index"))

## Global constants

In [ ]:
# Define constants
FOOD_NAME = "food name"
FOOD_IMAGE = "food image"
FOOD_IMAGE_PATH = "food image path"
FOOD_INGREDIENT = "food ingredient"
FOOD_INSTRUCTION = "food instruction"

# Dataset
NAME = "name"
COOK_TIME = "cook_time"
PREP_TIME = "prep_time"
DESCRIPTION = "description"
RECIPE_CATEGORY = "recipe_category"
INGREDIENT_QUANTITIES = "ingredient_quantites"
INGREDIENT_PARTS = "ingredient_parts"
RATING = "rating"
CALORIES = "calories"
FAT = "fat"
SATURATED_FAT = "saturated_fat"
CHOLESTEROL = "cholesterol"
SODIUM = "sodium"
CARBOHYDRATE = "carbohydrate"
FIBER = "fiber"
SUGAR = "sugar"
PROTEIN = "protein"
RECIPE_SERVING = "recipe_serving"
RECIPE_YIELD = "recipe_yield"
RECIPE_INSTRUCTION = "recipe_instruction"
ALLERGY = "allergy"

# Task
CALORIES_CALCULATOR = "nutrition"
COOK_HELPER = "recipe provider"
ALLERGEN_FILTER = "allergen avoidance"
FOOD_PREFERENCE = "food tag"
MEAL_PLANNER = "diet meal planner"

LEFTOVER_REMIXER = "recipe from ingredient"
DIETARY_FILTER = "dietary filter"
UNSPECIFIED = "unspecified"
UNCLASSIFIED = "unclassified"

TASK_LIST = [CALORIES_CALCULATOR, ALLERGEN_FILTER, FOOD_PREFERENCE, COOK_HELPER, MEAL_PLANNER]

# Model
TASK_CLASSIFIER_MODEL_NAME = "facebook/bart-large-mnli"
#TASK_CLASSIFIER_THRESHOLD = 0.5
TASK_CLASSIFIER_THRESHOLD = 0.65
RAG_DISTANCE_THRESGHOLD = 0.65

CLIPMODEL_NAME = "openai/clip-vit-base-patch32"

SENTENCE_MINI_MODEL = "all-MiniLM-L6-v2"
SENTENCE_BEST_MODEL = "all-mpnet-base-v2"



# Path
SAVE_IMAGE_PATH = Path("/content/images")
SAVE_EMBEDDED_DATA_PATH = Path("/content/embedding")
SAVE_INDEX_PATH = Path("/content/faiss_index")
FOOD_DATABASE_PATH = Path("/content/drive/MyDrive/nlp_database/food_database.json")
NLP_DATABASE = Path("/content/drive/MyDrive/nlp_database")

# Faiss
K_ORDER = 5



In [ ]:
from google.colab import drive
drive.mount("/content/drive")

Mounted at /content/drive


In [ ]:
import json
from pathlib import Path

In [ ]:
# load image dataset
import kagglehub

# Download latest version
KAGGLE_PATH = kagglehub.dataset_download("kmader/food41")

# print("Path to dataset files:", path)


## Global Models

In [ ]:
task_classifier_pipeline = pipeline("zero-shot-classification", model=TASK_CLASSIFIER_MODEL_NAME)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/1.15k [00:00<?, ?B/s]

Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


model.safetensors:   0%|          | 0.00/1.63G [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

Device set to use cuda:0


In [ ]:
clip_model = CLIPModel.from_pretrained(CLIPMODEL_NAME).to(get_device())
clip_processor = CLIPProcessor.from_pretrained(CLIPMODEL_NAME)

config.json:   0%|          | 0.00/4.19k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/605M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/605M [00:00<?, ?B/s]

Using a slow image processor as `use_fast` is unset and a slow processor was saved with this model. `use_fast=True` will be the default behavior in v4.52, even if the model was saved with a slow processor. This will result in minor differences in outputs. You'll still be able to use a slow processor with `use_fast=False`.


preprocessor_config.json:   0%|          | 0.00/316 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/592 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/862k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/525k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.22M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/389 [00:00<?, ?B/s]

In [ ]:
sentences_model = SentenceTransformer(SENTENCE_MINI_MODEL).to(get_device())

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.5k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/612 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [ ]:


reranking_model = CrossEncoder('cross-encoder/ms-marco-MiniLM-L-6-v2')


In [ ]:
# load llava 1.6
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.float16,
)

llm_model = LlavaNextForConditionalGeneration.from_pretrained(
    "llava-hf/llava-v1.6-mistral-7b-hf",
    quantization_config=bnb_config,
    device_map="auto"
)
llm_processor = AutoProcessor.from_pretrained("llava-hf/llava-v1.6-mistral-7b-hf")

config.json:   0%|          | 0.00/1.28k [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/70.2k [00:00<?, ?B/s]

Fetching 4 files:   0%|          | 0/4 [00:00<?, ?it/s]

model-00001-of-00004.safetensors:   0%|          | 0.00/4.92G [00:00<?, ?B/s]

model-00003-of-00004.safetensors:   0%|          | 0.00/4.92G [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/2.07k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/493k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.80M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/41.0 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/552 [00:00<?, ?B/s]

## 1. Implement function classify task



In [ ]:
# @title
# Load the CLIP model and processor



service_labels = ["Calories Calculator", "Cook Helper", "Dietary Filter"]

def get_calories():
    return "Calories Calculator Service!"

def get_recipe():
    return "Cook Helper Service: Finding a recipe...!"

def get_dietary_info():
    return "Dietary Filter Service: Checking allergens and dietary tags...!"

def classify_input(text_input, image_input):
    """
    Classify input into Calories Calculator, Cook Helper, or Dietary Filter.
    Requires both text and image input.
    """
    if not text_input or not image_input:
        raise ValueError("Both text and image are required for classification.")

    # Preprocessing input (text + image)
    inputs = clip_processor(
        text=[text_input] * len(service_labels),
        images=image_input,
        return_tensors="pt",
        padding=True
    )

    # Get image-text similarity logits outputs
    outputs = clip_model(**inputs)

    # Zero-shot classification probabilities
    probs = outputs.logits_per_image.softmax(dim=1)

    # Find the most probable service
    predicted_idx = torch.argmax(probs, dim=1).item()
    predicted_service = service_labels[predicted_idx]

    return predicted_service

def keyword_bias(text_input):
    text = text_input.lower()

    if any(word in text for word in ["recipe", "ingredients", "cook", "steps", "how to make"]):
        return "Cook Helper"
    elif any(word in text for word in ["calories", "nutrition", "macros", "vitamins"]):
        return "Calories Calculator"
    elif any(word in text for word in ["allergen", "vegan", "keto", "substitute"]):
        return "Dietary Filter"
    else:
        return None

def route_service(text_input, image_input):

    # Check keywords first
    keyword_prediction = keyword_bias(text_input)

    if keyword_prediction:
        predicted_service = keyword_prediction
    else:
        predicted_service = classify_input(text_input, image_input)

    if predicted_service == "Calories Calculator":
        return get_calories()
    elif predicted_service == "Cook Helper":
        return get_recipe()
    elif predicted_service == "Dietary Filter":
        return get_dietary_info()
    else:
        return "Service not found."


In [ ]:
# @title
def classify_tasks(query_text: str, debug: bool = False) -> list[str]:
  result = task_classifier_pipeline(query_text, TASK_LIST, multi_label=True)
  if debug:
    print(result)
  tasks = []
  for i, score in enumerate(result["scores"]):
    if score > TASK_CLASSIFIER_THRESHOLD:
      tasks.append(result["labels"][i])

  # handle the unknown task
  if not tasks or (COOK_HELPER not in tasks and CALORIES_CALCULATOR not in tasks and MEAL_PLANNER not in tasks):
    tasks.append(UNCLASSIFIED)

  return tasks


Improved version of classify task

1.Preprocesses the query text (lowercase and strip whitespace)

2.Uses keyword matching in addition to the NLI model

3.Adds contextual rules to catch specific scenarios (like leftover ingredients)



In [ ]:
# @title
#test1
def classify_tasks(query_text: str, debug: bool = False) -> list[str]:
    # Preprocess the query text

    query_text = query_text.lower().strip()
    query_text = re.sub(r"[^\w\s]", "", query_text) #remove punctuation

    # Create a set of keywords for each task
    task_keywords = {
        CALORIES_CALCULATOR: ["calorie", "calories", "nutrition", "nutritional", "count", "calculate", "tracker", "how many", "macros"],
        ALLERGEN_FILTER: ["allergy", "allergen", "allergic", "intolerance", "without", "free from", "avoid", "can't eat"],
        FOOD_PREFERENCE: ["like", "prefer", "favorite", "vegan", "vegetarian", "pescatarian", "keto", "paleo", "gluten-free", "dairy-free", "diet","creative uses","idea","ideas"],
        COOK_HELPER: ["recipe", "cook", "make", "prepare", "how to", "instruction", "dish", "meal", "dinner", "lunch", "breakfast"]
    }

    # Use the existing NLI model for classification
    result = task_classifier_pipeline(query_text, TASK_LIST, multi_label=True)

    if debug:
        print("NLI Model Classification:", result)

    # Get tasks from NLI model
    tasks = []
    for i, score in enumerate(result["scores"]):
        if score > TASK_CLASSIFIER_THRESHOLD:
            tasks.append(result["labels"][i])

    # Enhance with keyword matching
    for task, keywords in task_keywords.items():
        if task not in tasks:  # Only check if not already classified
            for keyword in keywords:
                if keyword in query_text:
                    tasks.append(task)
                    break

    # Add contextual rules
    if any(word in query_text for word in ["leftover", "ingredients", "have", "fridge", "remaining"]):
        if LEFTOVER_REMIXER not in tasks:
            tasks.append(LEFTOVER_REMIXER)

    if any(phrase in query_text for phrase in ["can't eat", "allergic to", "no dairy", "no nuts", "no gluten","cannot eat"]):
        if ALLERGEN_FILTER not in tasks:
            tasks.append(ALLERGEN_FILTER)

    # Handle the unknown task
    if not tasks:
        tasks.append(UNCLASSIFIED)

    #if debug:
       # print("Final classification:", tasks)

    return tasks

In [ ]:
# @title
from transformers import T5ForConditionalGeneration, T5Tokenizer

model = T5ForConditionalGeneration.from_pretrained("t5-small")
tokenizer = T5Tokenizer.from_pretrained("t5-small")

input_text = "Generate a hamburger recipe without tomatoes: "
inputs = tokenizer(input_text, return_tensors="pt")
outputs = model.generate(**inputs)
print(tokenizer.decode(outputs[0], skip_special_tokens=True))

config.json:   0%|          | 0.00/1.21k [00:00<?, ?B/s]

Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


model.safetensors:   0%|          | 0.00/242M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/2.32k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.39M [00:00<?, ?B/s]

You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565


Generate a hamburger recipe without tomatoes:


### Test classify tasks function

In [ ]:
classify_tasks("Give me a diet plan", True)

{'sequence': 'Give me a diet plan', 'labels': ['diet meal planner', 'food tag', 'nutrition calculator', 'allergen avoidance', 'recipe provider'], 'scores': [0.8111365437507629, 0.14289841055870056, 0.10950033366680145, 0.048765432089567184, 0.009072900749742985]}


['diet meal planner']

In [ ]:
# @title

user_data = {
    "text": None,
    "image": None
}

if __name__ == "__main__":
    # Ask user to input text + image
    text = input("Enter your question about the food: ")
    user_data["text"] = text

    image = input("Upload your food image: ")     # use image path
    user_data["image"] = Image.open(image)

    # Get service classification
    output = route_service(text, image)
    print(output)
    # print(f"\nText: {user_data['text']}")


KeyboardInterrupt: Interrupted by user

In [ ]:
# @title
# Test

query_text_list = ["Can you give me a meal from these leftover ingredients: fish, tomato, pasta consider that I am allergic to dairy product",
                   "I need a quick dinner recipe using chicken and broccoli, no gluten please",
                   "Breakfast ideas with eggs that are keto-friendly",
                   "What can I cook with rice and beans that is vegan?",
                   "Dessert recipe without sugar as I have diabetes",
                   "What is the calories for this food?",
                   "Can I eat this food?",
                   "Give me the any recipe that contains meat but no vegetable",
                   "Give me the any recipe that contains meat but I am allergic to diary product",
                   "I want to know the calories and recipe of pizza",
                   "Breakfast idea with eggs but I am allergic to milk",
                   "Breakfast idea with eggs",
                   "UWU"]
if __name__ == "__main__":
    # Ask user to input text + image
    # text = input("Enter your question: ")
    # Get service classification
    for query_text in query_text_list:
      tasks = classify_tasks(query_text, True)
      print(query_text)
      print(tasks)
      print('-------------')

In [ ]:
# @title
#test2

query_text_list = [
    # LEFTOVER_REMIXER examples
    "What can I make with chicken, bell peppers, and onions in my fridge?",
    "I have leftover rice, beans, and corn. What can I cook?",
    "Help me use up these ingredients: pasta, tuna, and olives",
    "Recipe ideas for remaining thanksgiving turkey",
    "Creative uses for stale bread",

    # COOK_HELPER examples
    "How do I make chicken parmesan?",
    "Simple pasta recipe for beginners",
    "Best way to cook salmon",
    "Teach me how to bake chocolate chip cookies",
    "Quick dinner ideas for a family of four",

    # CALORIES_CALCULATOR examples
    "How many calories in a banana?",
    "Nutritional value of avocado toast",
    "Count calories in homemade lasagna",
    "What's the fat content in a croissant?",
    "Protein in a 6oz steak",

    # ALLERGEN_FILTER examples
    "Desserts without dairy",
    "Gluten-free bread recipes",
    "What can I eat if I'm allergic to shellfish?",
    "Nut-free snack ideas for kids",
    "Recipes suitable for someone with egg allergy",

    # FOOD_PREFERENCE examples
    "Vegan dinner ideas",
    "Keto-friendly breakfast options",
    "Mediterranean diet lunch recipes",
    "Low-carb meals for the week",
    "High protein vegetarian dinners",

    # Multi-task examples
    "Calculate calories in a pasta dish without gluten",
    "Vegan recipes using only ingredients I have: tofu, broccoli, and rice",
    "How to make a low-calorie cheesecake for someone with nut allergies",
    "Keto breakfast ideas using eggs and avocado I have in my fridge",
    "What's the nutritional breakdown of a dairy-free vegetable soup?",

    # Edge cases
    "Is this food safe to eat after 3 days?",
    "Can dogs eat chocolate?",
    "Where can I order pizza near me?",
    "Food photography tips",
    "History of Italian cuisine",

    # Ambiguous queries
    "Food ideas",
    "Help with dinner",
    "I'm hungry",
    "What should I eat?",
    "Best meals",

    # Nonsensical queries
    "XYZ123",
    "Hello world",
    "The sky is blue",
    "Dancing elephant",
    "UwU"
]

if __name__ == "__main__":
    # Test each query
    for query_text in query_text_list:
        tasks = classify_tasks(query_text, True)
        print(f"Query: {query_text}")
        print(f"Classification: {tasks}")
        print('-------------')

## 2. Implement function to separate intent and constraint


In [ ]:
# improve the filter pattern later
def separate_intent_and_constraint(user_text: str) -> tuple[str, list[str]]:
    """
    Function to separate constriant of the query from the intention query.
    This function should be used to only the query that contains Dietary Filter Task
    """

    constraint_keywords = ["but", "without", "avoid", "except", "no", "must be", "should be"]

    lower_text = user_text.lower()

    for keyword in constraint_keywords:
        if keyword in lower_text:
            parts = re.split(rf"\b{keyword}\b", user_text, maxsplit=1, flags=re.IGNORECASE)
            intent_query = parts[0].strip()
            constraint = parts[1].strip() if len(parts) > 1 else ""

            intent_query = intent_query.rstrip(",.?!").rstrip()
            intent_query = re.sub(r'\b(with|without|but|except|avoid|no|must be|should be)\b$', '', intent_query, flags=re.IGNORECASE).strip()

            constraint = constraint.strip('.,?')
            return intent_query, [constraint]

    return user_text.strip(), ""


In [ ]:
# @title
from transformers import pipeline

ner = pipeline("ner", model="dslim/bert-base-NER")
query = "I want a hamburger recipe without tomatoes"

constraint_entities = [entity["word"] for entity in ner(query) if entity["entity"] == "B-MISC"]
print("Constraints:", constraint_entities)  # ['tomatoes']

config.json:   0%|          | 0.00/829 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/433M [00:00<?, ?B/s]

Some weights of the model checkpoint at dslim/bert-base-NER were not used when initializing BertForTokenClassification: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight']
- This IS expected if you are initializing BertForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


tokenizer_config.json:   0%|          | 0.00/59.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/213k [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/2.00 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

Device set to use cuda:0


Constraints: []


### Test separate intent and constraint function

In [ ]:
# @title
if __name__ == "__main__":
    user_input = user_data["text"]
    intent, constraint = separate_intent_and_constraint(user_input)

    print(f"\nIntent Query: {intent}")
    print(f"Constraint: {constraint}")


Intent Query: Give me the recipes of this burger
Constraint: chesses


## 3. Preprocess the datasets

### Merging data
Currently, we merge datasets in a sqlite database by joining them on a food_name column. Then export to a json file.

In [ ]:
# create metadata and rich text from merged datasets

def get_metadata_dict_list(json_database_path: Path) -> list[dict[str, list[str] | str]]:
  if not os.path.exists(json_database_path):
    raise FileNotFoundError(f"File not found: {json_database_path}")

  with open(json_database_path, "r", encoding="utf-8") as f:
    metadata_dict_list = json.load(f)

  def process_column(metadata_dict: dict[str,str]):
    for key, value in metadata_dict.items():
      if value is None:
        metadata_dict[key] = "missing" # TODO: Come up with a better text to let llm know that they have to search if it encounters this text
      elif key == ALLERGY:
        metadata_dict[key] = value.split(", ")
      elif key == INGREDIENT_QUANTITIES:
        try:
          metadata_dict[key] = eval(value.replace("NA", '"missing"'))
        except:
          metadata_dict[key] = "missing"
      elif key in [INGREDIENT_PARTS, RECIPE_INSTRUCTION]:
        try:
          metadata_dict[key] = eval(value)
        except:
          metadata_dict[key] = "missing"
    return metadata_dict

  # to process the str list to list and fill the missing value for llm to search
  metadata_dict_list = list(map(process_column, metadata_dict_list))

  return metadata_dict_list

def create_rich_text(metadata_dict_list: dict[str, list[str] | str]) -> list[str]:
  rich_text_list = []

  # construct rich text to store in the vector database
  for metadata_dict in tqdm(metadata_dict_list):
    if metadata_dict is None:
      continue
    rich_text_list.append(
        f"Recipe: {metadata_dict[NAME]}. Dish type: {metadata_dict[RECIPE_CATEGORY]}. Main ingredients: {', '.join(metadata_dict[INGREDIENT_PARTS])}, Serving: {metadata_dict[RECIPE_SERVING]} people. Review rating: {metadata_dict[RATING]}. Cook time: {metadata_dict[COOK_TIME]}. Preparation time: {metadata_dict[PREP_TIME]}. Nutrition - Calories: {metadata_dict[CALORIES]}, Fat: {metadata_dict[FAT]}, Sodium: {metadata_dict[SODIUM]}, Cholesterol: {metadata_dict[CHOLESTEROL]}, Sugar: {metadata_dict[SUGAR]}, Protein: {metadata_dict[PROTEIN]}."
        )

  return rich_text_list


In [ ]:
# save image and create image metadata
def create_food_image_metadata_dict_list(path) -> list[str]:
  food_image_metadata_dict_list = []
  path_test = path + "/meta/meta/test.txt"
  if os.path.exists(path_test):
    with open(path_test, 'r') as file:
        content = file.read()
  # construct rich image to store in the vector database
  for sub in tqdm(content.split('\n')):
    if sub is None:
      continue
    elif sub == "":
      continue
    path_image = path + '/images/' + sub + '.jpg'
    if os.path.exists(path_image):
      food_image_metadata_dict_list.append({
          NAME: sub.split('/')[0],
          FOOD_IMAGE_PATH: path_image
      })
    else:
      print(sub)

  return food_image_metadata_dict_list

In [ ]:
# @title

image_dataset_path = Path("/kaggle/input/food41/images")

def generate_metadata(image_dir: Path) -> list[dict]:
    return [
        {
            "image_path": str(img),
            "dish_name": label.name.replace('_', ' ').title(),
            "category": label.name,
            "ingredients": [],
            "recipe": "Recipe description...",
            "cooking_time": "30 mins",
            "nutrition": {"calories": 250, "fat": 10, "protein": 8, "sodium": 300}
        }
        for label in sorted(image_dir.iterdir()) if label.is_dir()
        for img in label.glob("*.jpg")
    ]

def create_rich_image_list(metadata: list[dict]) -> list[dict]:
    return [
        {
            "image_path": item["image_path"],
            "rich_text": (
                f"Dish Name: {item['dish_name']}. "
                f"Ingredients: {', '.join(item['ingredients']) if item['ingredients'] else 'N/A'}. "
                f"Cooking time: {item['cooking_time']}. "
                f"Category: {item['category']}. "
                f"Nutrition: Calories: {item['nutrition']['calories']} kcal, "
                f"Fat: {item['nutrition']['fat']}g, "
                f"Protein: {item['nutrition']['protein']}g, "
                f"Sodium: {item['nutrition']['sodium']}mg."
            )
        }
        for item in tqdm(metadata)
    ]

metadata_list = generate_metadata(image_dataset_path)
rich_image_list = create_rich_image_list(metadata_list)

for item in rich_image_list:
    print(item)


### Test create rich text and metadata

In [ ]:
main_metadata_dict_list = get_metadata_dict_list(FOOD_DATABASE_PATH)

In [ ]:
with open(NLP_DATABASE / "food_allergens.json", "r", encoding="utf-8") as f:
    food_allergen_table = json.load(f)

food_allergen_table

[{'allergy': 'Nut Allergy', 'group': 'Oil seed', 'id': 185, 'name': 'Almond'},
 {'allergy': 'Oral Allergy Syndrome',
  'group': 'Pome fruit',
  'id': 186,
  'name': 'Apple'},
 {'allergy': 'Stone Fruit Allergy',
  'group': 'Stone fruit',
  'id': 187,
  'name': 'Apricot'},
 {'allergy': 'Insulin Allergy',
  'group': 'Composite vegetable',
  'id': 188,
  'name': 'Artichoke'},
 {'allergy': 'Allium Allergy',
  'group': 'Liliaceous vegetable',
  'id': 189,
  'name': 'Asparagus'},
 {'allergy': 'Oral Allergy Syndrome',
  'group': 'Tropical fruit',
  'id': 190,
  'name': 'Avocado'},
 {'allergy': 'Histamine Allergy',
  'group': 'Miscellaneous vegetable',
  'id': 191,
  'name': 'Bamboo shoot'},
 {'allergy': 'Banana Allergy',
  'group': 'Tropical fruit',
  'id': 192,
  'name': 'Banana'},
 {'allergy': 'Gluten Allergy',
  'group': 'Cereal grain',
  'id': 193,
  'name': 'Barley'},
 {'allergy': 'Legume Allergy', 'group': 'Pulse', 'id': 194, 'name': 'Bean'},
 {'allergy': 'Salicylate Allergy',
  'group':

In [ ]:
food_allergy_list = [metadata["allergy"].strip() for metadata in food_allergen_table] + [metadata["name"].strip() for metadata in food_allergen_table]
food_allergy_list

['Nut Allergy',
 'Oral Allergy Syndrome',
 'Stone Fruit Allergy',
 'Insulin Allergy',
 'Allium Allergy',
 'Oral Allergy Syndrome',
 'Histamine Allergy',
 'Banana Allergy',
 'Gluten Allergy',
 'Legume Allergy',
 'Salicylate Allergy',
 'Legume Allergy',
 'Salicylate Allergy',
 'Histamine Allergy',
 'Legume Allergy',
 'Broccoli allergy',
 'Cruciferous Allergy',
 'Gluten Allergy',
 'Ragweed Allergy',
 'Milk allergy / Lactose intolerance',
 'Legume Allergy',
 'Milk allergy / Lactose intolerance',
 'Mushroom Allergy',
 'Cruciferous Allergy',
 'Cacao bean',
 'Canola oil',
 'Hypersensitivity',
 'Milk allergy / Lactose intolerance',
 'Alpha-gal Syndrome',
 'Cruciferous Allergy',
 'Hypersensitivity',
 'Milk allergy / Lactose intolerance',
 'Stone Fruit Allergy',
 'Nut Allergy',
 'Poultry Allergy',
 'Insulin Allergy',
 'Cruciferous Allergy',
 'Coconut oil',
 'Ochratoxin Allergy',
 'Corn Allergy',
 'Corn oil',
 'Seed Allergy',
 'Salicylate Allergy',
 'Milk allergy / Lactose intolerance',
 'Shellfi

In [ ]:
food_allergy_list = list(set(food_allergy_list))
print(len(food_allergy_list))
food_allergy_list

222


['Sugar beet',
 'Welsh',
 'Kyona',
 'Percifomes',
 'Thyroid',
 'Burdock',
 'Canola oil',
 'Makuwauri  melon',
 'Mineral water',
 'Watercress',
 'Grapefruit',
 'Rice',
 'Lemon',
 'Chinese cabbage',
 'Eel',
 'Grape',
 'Sweet potato',
 'Hypersensitivity',
 'Deer',
 'Mango',
 'Bonito',
 'Cabbage',
 'Legume Allergy',
 'Mitsuba',
 'Endive',
 'Soy Allergy',
 'Peanut Allergy',
 'Cacao bean',
 'Bamboo shoot',
 'Buttermilk',
 'Glucose',
 'Sour cream',
 'Yam',
 'Peppermint',
 'Ice cream',
 'Apricot',
 'Pear',
 'Kidney bean',
 'Insulin Allergy',
 'Sheep',
 'Goat',
 'Butter',
 'Horse Mackerel',
 'Walnut',
 'Soybean',
 'Tea',
 'Spearmint',
 'Cream',
 'Horseradish',
 'Chestnut',
 'Blackberry',
 'Sea Bream',
 'Asparagus',
 'Duck',
 'Salsify',
 'Radish root',
 'Bean',
 'Pecan',
 'Orange',
 'Hop',
 'Japanese plum',
 'Allium Allergy',
 'Lentil',
 'Nira',
 'Gluten Allergy',
 'Garlic',
 'Peas',
 'Passion fruit',
 'Sweet Pepper',
 'Strawberry',
 'Histamine Allergy',
 'Apple',
 'Pumpkin',
 'White bean',
 'Da

In [ ]:
food_allergy_list += [metadata[""]]

### Test create rich image and image label metadata

In [ ]:
# save image and create image metadata
food_image_metadata_dict_list = create_food_image_metadata_dict_list(KAGGLE_PATH)

100%|██████████| 25251/25251 [00:00<00:00, 216219.95it/s]


In [ ]:
# @title
rich_image_label_for_sentences = [food_image_metadata_dict[NAME] for food_image_metadata_dict in food_image_metadata_dict_list]

In [ ]:
# @title
rich_text_list = create_rich_text(metadata_dict_list)

100%|██████████| 522517/522517 [00:02<00:00, 197599.53it/s]


## 4. Implement RAG vector database system wint fine-tuned multimodal embedding + faiss

In [ ]:
# embedding rich text and image

batch_size = 16
batch_save = 250

def get_embedded_data(data_list: list[str | Image.Image] | str | Image.Image, embedding_model: str, resume_index: int = 0, save: bool = False) -> torch.Tensor:
  embedded_data_list = []

  if not isinstance(data_list, list):
    data_list = [data_list]

  Path("/content/save_embed/").mkdir( parents=True, exist_ok=True)

  data_type = "image" if isinstance(data_list[0], Image.Image) else "text"se
  print(data_type)
  if not os.path.exists(SAVE_EMBEDDED_DATA_PATH ):
    SAVE_EMBEDDED_DATA_PATH.mkdir( parents=True, exist_ok=True)
  try:

    for i in tqdm(range(resume_index, math.ceil(len(data_list) / batch_size))):
      spliced_data_list = data_list[i * batch_size : min(len(data_list), (i + 1) * batch_size)]
      if embedding_model == "clip":
        if data_type == "image":
          processed_data_inputs = clip_processor(images=spliced_data_list, return_tensors="pt", padding=True, truncation=True)

          processed_data_inputs = {k : v.to(get_device()) for k,v in processed_data_inputs.items()}

          embedded_data_list.extend(clip_model.get_image_features(**processed_data_inputs).detach().cpu().numpy())
        elif data_type == "text":
          processed_data_inputs = clip_processor(spliced_data_list, return_tensors="pt", padding=True, truncation=True)

          processed_data_inputs = {k : v.to(get_device()) for k,v in processed_data_inputs.items()}
          embedded_data_list.extend(clip_model.get_text_features(**processed_data_inputs).detach().cpu().numpy())

      elif embedding_model == "sentence":
        embedded_data_list.extend(sentences_model.encode(spliced_data_list))

      if save and i % batch_save == 0 and i != 0:
        np.save(Path("/content/save_embed") / f"{data_type}_{embedding_model}_{time.time()}_{i}.npy", np.array(embedded_data_list))
        with open("save_log.txt", "a") as f:
          f.write(f"save: type: {data_type}. start embedding from index: {resume_index}, end index: {i}, time: {time.time()}\n")


    embedded_data_array = np.array(embedded_data_list)

    # save embedded text
    if save:
      np.save(SAVE_EMBEDDED_DATA_PATH / f"{data_type}_{embedding_model}.npy", embedded_data_array)

  except KeyboardInterrupt:
    print("keyboard interupt")
  except Exception as e:
    print(e)
    embedded_data_array = np.array(embedded_data_list)

    Path("/content/interupt_save").mkdir( parents=True, exist_ok=True)

    np.save(Path("/content/interupt_save") / f"{data_type}_{embedding_model}_interupt_{time.time()}.npy", embedded_data_array)

    with open("error_log.txt", "a") as f:
      f.write(f"error: type: {data_type}. start embedding from index: {resume_index}, end index: {i}, time: {time.time()}\n")

  return embedded_data_array




In [ ]:
test = get_embedded_data(Image.open("/content/carbonara.jpg"), 'clip')

image


100%|██████████| 1/1 [00:00<00:00,  1.09it/s]


In [ ]:
# Create index using faiss
def get_faiss_index(embedded_data: np.ndarray, method: str = "flat", save: bool = False, name: str = "") -> faiss.IndexFlatL2:
  index = faiss.IndexFlatL2(embedded_data.shape[1]) # change to HNWS
  index.add(embedded_data)

  if save:
    Path("/content/faiss_index").mkdir( parents=True, exist_ok=True)
    faiss.write_index(index, str(Path("/content/faiss_index") / f"{name}_index_{time.time()}.index"))

  return index

In [ ]:
# Faiss similarity search
def faiss_search(index: faiss.IndexFlatL2, embbeded_query: np.ndarray, k: int = K_ORDER) -> tuple[np.ndarray, np.ndarray]:
  distance_array, indice_array = index.search(embbeded_query, k)
  return distance_array[0], indice_array[0]

### Test full RAG


#### Create / Load main_index

In [ ]:
# @title
response = requests.get("https://upload.wikimedia.org/wikipedia/commons/thumb/0/0b/RedDot_Burger.jpg/500px-RedDot_Burger.jpg")
img = Image.open(BytesIO(response.content))
display(img)

In [ ]:
# @title
from io import BytesIO
# define query
query_text = "apple pie"

url = "https://www.primaverakitchen.com/wp-content/uploads/2024/08/Hamburger-Recipe-7.jpg"
query_image = img

# encode
# process query
processed_query_inputs = clip_processor(text=query_text, images=query_image, return_tensors="pt", padding=True)

# encode processed querry
query_output = clip_model(**processed_query_inputs)

query_text_embedding = query_output.text_embeds.detach().cpu().numpy()
query_image_embedding = query_output.image_embeds.detach().cpu().numpy()

In [ ]:
# embed rich text

In [ ]:
# @title
# embedding rich text
text_embbedding = get_embedded_data(rich_text_list, "sentence", 0, True)

In [ ]:
# @title
# alternative load data
text_embbedding = load_embedded_data("text")

In [ ]:
# @title
# get faiss index
text_index = get_faiss_index(text_embbedding, "flat", True, "main")

In [ ]:
# alterbnavtie load index
main_index = load_faiss_index("main_index_1745901394.7850068")

In [ ]:
# get embedded querry
embedded_query_text = get_embedded_data("I want any sandwich recipe", "sentence")

text


100%|██████████| 1/1 [00:00<00:00,  1.43it/s]


In [ ]:
distance_array, indice_array = faiss_search(main_index, embedded_query_text)

In [ ]:
for indice, distance in zip(indice_array, distance_array):
  print(f"distance: {distance}")
  print("metadata:")
  print(rich_text_list[indice])
  print("---------------------------------")

distance: 0.44974276423454285
metadata:
Recipe: Sandwich for 12. Dish type: Lunch/Snacks. Main ingredients: old fashioned oats, boiling water, butter, margarine, eggs, egg, sesame seeds, mayonnaise, prepared mustard, lettuce leaf, cooked ham, green pepper, onion, swiss cheese, tomatoes, Serving: 12.0 people. Review rating: missing. Cook time: 0:30:00. Preparation time: 1:00:00. Nutrition - Calories: 378.1, Fat: 18.3, Sodium: 520.3, Cholesterol: 97.6, Sugar: 6.6, Protein: 20.5.
---------------------------------
distance: 0.45544010400772095
metadata:
Recipe: Best Breakfast Sandwich. Dish type: One Dish Meal. Main ingredients: butter, mayonnaise, eggs, water, milk, salt, pepper, deli ham, American cheese, swiss cheese, onion, tomatoes, Serving: 1.0 people. Review rating: missing. Cook time: 0:10:00. Preparation time: 0:05:00. Nutrition - Calories: 428.1, Fat: 27.8, Sodium: 1131.8, Cholesterol: 494.3, Sugar: 2.8, Protein: 34.1.
---------------------------------
distance: 0.525003552436828

#### Create / Load  food_image_index

In [ ]:
# @title
embedded_food_label_clip = get_embedded_data(rich_image_label_for_sentences, "clip", 0, True)

text


100%|██████████| 1579/1579 [00:28<00:00, 54.60it/s]


In [ ]:
embedded_food_image_clip = load_embedded_data("image_clip")

In [ ]:
food_image_index = get_faiss_index(embedded_food_image_clip, "flat", True, "food_image_clip")

In [ ]:
embedded_query_text = get_embedded_data("Pizza", "clip")

In [ ]:
distance_array, indice_array = faiss_search(food_image_index, embedded_query_text)

In [ ]:
for indice, distance in zip(indice_array, distance_array):
  print(f"distance: {distance}")
  print("metadata:")
  print(food_image_metadata_dict_list[indice])
  print("---------------------------------")

#### Create / Load food_label_index

In [ ]:
food_label_index = load_faiss_index("food_label_clip_index_1745943408.8384023")

In [ ]:
food_label_metadata_dict_list = food_image_metadata_dict_list

#### Create / Load food_label_sentence_index

In [ ]:
embedded_food_label_sentence = get_embedded_data(rich_image_label_for_sentences, "sentence", 0, True)

text


100%|██████████| 1579/1579 [00:19<00:00, 79.90it/s]


In [ ]:
food_label_sentence_index = get_faiss_index(embedded_food_label_sentence, "flat", True, "food_label_sentence")

In [ ]:
food_label_sentence_index = load_faiss_index("food_label_sentence_index_1745943412.1561074")

In [ ]:
embedded_query_text = get_embedded_data("Pizza", "sentence", 0, False)

In [ ]:
distance_array, indice_array = faiss_search(food_label_sentence_index, embedded_query_text)

In [ ]:
for indice, distance in zip(indice_array, distance_array):
  print(f"distance: {distance}")
  print("metadata:")
  print(food_image_metadata_dict_list[indice])
  print("---------------------------------")

## 5. Construct prompt for the llm

In [ ]:
def process_retrieved_text(
                           indice_array: np.ndarray, distance_array: np.ndarray,
                           tasks: list[str],
                           constraint_list:list[str]) -> tuple[list[str], bool]: # implement the constraint to be accepted as list

  text_content_list = []
  should_end = False
  is_filtering_allergy = ALLERGEN_FILTER in tasks or FOOD_PREFERENCE in tasks

  for i, indice in enumerate(indice_array):
      main_metadata_dict = main_metadata_dict_list[indice]

      if distance_array[i] > RAG_DISTANCE_THRESGHOLD:
        should_end = True
        break

      if is_filtering_allergy and constraint:
        allergy = False
        for constraint in constraint_list:
          if (
              str(main_metadata_dict[INGREDIENT_PARTS]).lower().find(constraint.lower()) != -1 or
              str(main_metadata_dict[ALLERGY]).lower().find(constraint.lower()) != -1
              ):
            allergy = True
            break

        if allergy:
            continue


      text_content.append(f"""[**Food Name**: {main_metadata_dict[NAME]}
**User Rating**: {main_metadata_dict[RATING]} stars
**Food Category**: {main_metadata_dict[RECIPE_CATEGORY]}
**Cook Time**: {main_metadata_dict[COOK_TIME]}
**Preparation Time**: {main_metadata_dict[PREP_TIME]}
**Ingredient Parts**: {", ".join(main_metadata_dict[INGREDIENT_PARTS])}
**Ingredient Quantities**: {", ".join(main_metadata_dict[INGREDIENT_QUANTITIES])}
**Nutrition**:
- Calories: {main_metadata_dict[CALORIES]} cals
- Carbohydrate: {main_metadata_dict[CARBOHYDRATE]} g
- Protein: {main_metadata_dict[PROTEIN]} g
- Fat: {main_metadata_dict[FAT]} g
- Saturated Fat: {main_metadata_dict[SATURATED_FAT]} g
- Fiber: {main_metadata_dict[FIBER]} g
- Cholesterol: {main_metadata_dict[CHOLESTEROL]} mg
- Sodium: {main_metadata_dict[SODIUM]} mg
- Sugar: {main_metadata_dict[SUGAR]} g
**Serving**: {main_metadata_dict[RECIPE_SERVING]} people
**Recipe Instruction**: {", ".join(main_metadata_dict[RECIPE_INSTRUCTION])}
**Not Suitable for People Who has Allergy**: {", ".join(main_metadata_dict[ALLERGY])}]
\n\n""")


  return text_content, should_end

In [ ]:
def reiterate_discarded(embedded_query_text: str,
                        tasks: list[str],
                        constraint_list:list[str],
                        debug: bool = False) -> str:

  should_end = False
  current_discarded_count = 0
  current_non_discarded_count = 0

  while True:
    distance_array, indice_array = faiss_search(main_index, embedded_query_text, k= K_ORDER + current_discarded_count)

    text_content, should_end = process_retrieved_text(indice_array, distance_array, tasks, constraint)

    current_non_discarded_count = len(text_content)
    current_discarded_count = K_ORDER - len(text_content)

    if not (current_non_discarded_count < K_ORDER and not should_end):
      break

    if debug:
      print(f"current non discarded count: {current_non_discarded_count}")

  if not text_content:
    text_content = "There is no context. Please use your knowledge."

  return text_content

In [ ]:

def rerank_text_content(text_content, query_text):

    scores = model.predict([(query_text, text) for text in text_content])

    # Sort by score in descending order
    ranked_results = sorted(zip(text_content, scores), key=lambda x: x[1], reverse=True)

    sorted_text_content = [text for text_content, score in ranked_results]

    return sorted_text_content


In [ ]:
# @title
# construct prompt based on the task that user input
def get_prompt_based_on_task(query: str, image: Image.Image, text_context: str, tasks: list[str]) -> list[dict]:
  conversation = [
    {
      "role": "system",
      "content": [
        {"type": "text", "text": "You are a home cook chef and nutritionist assistant. Your primary function is to provide users with diet plans and meal preparation instructions, taking into account of the potential allergies the user may have."}
        ]
      }
    ]
  for task in tasks:
    if task in [CALORIES_CALCULATOR, COOK_HELPER, UNSPECIFIED, UNCLASSIFIED]:
      task_name = task
      break

  prompt_context = f"""Given the query: '{query}' and the following relevant snippets:

  {text_context}

  """

  if set([COOK_HELPER, CALORIES_CALCULATOR]).issubset(set(tasks)):
    prompt_context += f"""Recommend a recipe based on a given query and image using the provided data in the snippets. If there is not enough data in the snippets, you can use your knowledge. If the data is missing, you have to fill it in using your knowledge. The response must include rating, cooking time, ingredients, nutrition, and instructions. You also have to give a suggestion or opinion as a home cook chef for this food."""

  elif task_name == CALORIES_CALCULATOR:
    prompt_context += f"""Write the summary of the nutrition of this food based on the given query and image. If there is not enough data in the snippets, you can use your knowledge. If the data is missing, you have to fill it in using your knowledge. The response should includes ingredients, calories, fat, saturated fat, protein, cholesterol, sodium, fiber, sugar, protein. You also have to give a suggestion or opinion as a nutritionist for this food."""

  elif task_name == COOK_HELPER:
    prompt_context += f"""Recommend a recipe based on a given query and image using the provided data in the snippets. You have to be detailed with the ingredients and the instruction. If there is not enough data in the snippets, you can use your knowledge. If the data is missing, you have to fill it in using your knowledge. The response must include rating, cooking time, ingredients, and instructions. You also have to give a suggestion or opinion as a home cook chef for this food."""

  # TODO: test whether adding dietary filter prompt will improve the generation

  elif task_name == UNSPECIFIED:
    prompt_context += f"""Use the provided data in snippets and the image to summary and conclude. You also have to give an opinion as a nutritionist and home cook chef for the image and data in the snippets"""

  elif task_name == UNCLASSIFIED:
    prompt_context += f"""Answer the question from the given query. You can use the data in the snippets as references. If there is not enough data in the snippets, you can use your knowledge. If the data is missing, you have to fill it in using your knowledge. You also have to give an opinion as a nutritionist and home cook chef to the query and image"""


  conversation.append(
        {
          "role": "user",
          "content": [
              {"type": "image", "image": image},
              {"type": "text", "text": prompt_context}
          ]
        }
        )

  return conversation

In [ ]:
# construct prompt based on the task that user input
# TODO: handle more task later
  conversation = [
    {
      "role": "system",
      "content": [
        {"type": "text", "text": "You are a home cook chef and nutritionist assistant. Your primary function is to provide users with diet plans, nutrition, recipe, and meal preparation instructions."}
        ]
      }
    ]
def get_prompt_based_on_task_mistral(conversation: list[dict], query: str, image: Image, text_content: str, tasks: list[str]) -> list[dict]:
    #, taking into account of the potential allergies the user may have.
  for task in tasks:
    if task in [CALORIES_CALCULATOR, COOK_HELPER, MEAL_PLANNER, UNSPECIFIED, UNCLASSIFIED]:
      task_name = task
      break

  prompt_context = f"""Given the query: '{query}' and the following relevant snippets:

  {text_content}

  """

  # if set([COOK_HELPER, CALORIES_CALCULATOR]).issubset(set(tasks)):
  #   prompt_context += f"""Recommend a recipe based on a given query and image using the provided data in the snippets. If there is not enough data in the snippets, you can use your knowledge. If the data is missing, you have to fill it in using your knowledge. The response must include rating, cooking time, ingredients, nutrition, and instructions. You also have to give a suggestion or opinion as a home cook chef for this food."""
  if task_name == MEAL_PLANNER:
    prompt_context += """Answer the given query. You can use both your knowledge and data in the snippet. The response should be in a readable format for a meal planner. Then ask user what else do you want to know."""

  elif task_name == CALORIES_CALCULATOR:
    prompt_context += """Answer the given query. You can use both your knowledge and data in the snippet. The response should be in a readable format for nutrition summary. Then ask user what else do you want to know."""
    # """Write the summary of the nutrition of this food based on the given query and image. If there is not enough data in the snippets, you can use your knowledge. If the data is missing, you have to fill it in using your knowledge. The response should includes ingredients, calories, fat, saturated fat, protein, cholesterol, sodium, fiber, sugar, protein. You also have to give a suggestion or opinion as a nutritionist for this food."""

  elif task_name == COOK_HELPER:
    prompt_context += """Answer the given query. You can use both your knowledge and data in the snippets. The response should be in a readable format for recipe and home cook. Then ask user what else do you want to know."""
    # """Recommend a recipe based on a given query and image using the provided data in the snippets. You have to be detailed with the ingredients and the instruction. If there is not enough data in the snippets, you can use your knowledge. If the data is missing, you have to fill it in using your knowledge. The response must include rating, cooking time, ingredients, and instructions. You also have to give a suggestion or opinion as a home cook chef for this food."""

  # TODO: test whether adding dietary filter prompt will improve the generation

  elif task_name == UNSPECIFIED:
    prompt_context += f"""Use the provided data in snippets and the image to summary and conclude. You also have to give an opinion as a nutritionist and home cook chef for the image and data in the snippets. Then ask user what else do you want to know."""

  elif task_name == UNCLASSIFIED:
    prompt_context += f"""Answer the given query using your own knowledge or provided data in the snippets. The response should be in a readable format. Then ask user what else do you want to know."""
    #  You can use the data in the snippets as references. If there is not enough data in the snippets, you can use your knowledge. If the data is missing, you have to fill it in using your knowledge. You also have to give an opinion as a nutritionist and home cook chef to the query and image"""

  if image is not None:
    conversation.append(
          {
            "role": "user",
            "content": [
                {"type": "image"},
                {"type": "text", "text": prompt_context}
            ]
          }
          )
  else:
    conversation.append(
          {
            "role": "user",
            "content": [
                {"type": "text", "text": prompt_context}
            ]
          }
          )

  return conversation

In [ ]:
# @title
# construct prompt based on the task that user input
def get_prompt_based_on_task(query: str, image: Image.Image, text_context: str, tasks: list[str]) -> list[dict]:
  conversation = [
    {
      "role": "system",
      "content": [
        {"type": "text", "text": "You are a home cook chef and nutritionist assistant. Your primary function is to provide users with diet plans and meal preparation instructions, taking into account of the potential allergies the user may have."}
        ]
      }
    ]
  for task in tasks:
    if task in [CALORIES_CALCULATOR, COOK_HELPER, UNSPECIFIED, UNCLASSIFIED]:
      task_name = task
      break
# intent query, constraint | query = intent query + constraint
  if task_name == CALORIES_CALCULATOR:
    conversation.append(
      {
        "role": "user",
        "content": [
            {"type": "image", "image": Image.open(image_meta_data[IMAGE_SAVE_PATH])},
            {"type": "text", "text": f"""
              Given the query "{query}" and the following relevant snippets:

              {text_context}

              Calculate the total calories and macros for the food based on snippets above.
              Breakdown per ingredient and include totals.
              Use USDA-standard nutritional values.
            """}
        ]
      }
    )
  elif task_name == COOK_HELPER:
    conversation.append(
      {
        "role": "user",
        "content": [
            {"type": "image", "image": Image.open(image_meta_data[IMAGE_SAVE_PATH])},
            {"type": "text", "text": f"""
              Given the query "{query}" and the following relevant snippets:

              {text_context}

              Recommend a recipe using provided data in the snippets following the food category and avoids allergen ingredients from the query.
              Include cooking time, difficulty, and steps.
            """}
        ]
      }
    )
  else:
    conversation.append(
          {
            "role": "user",
            "content": [
                {"type": "image", "image": Image.open(image_meta_data[IMAGE_SAVE_PATH])},
                {"type": "text", "text": f"""
                  Given the query "{query}" and the following relevant snippets:

                  {text_context}

                  Please provide a concise and accurate answer to the query, incorporating relevant information from the provided snippets where available.
                """}
            ]
          }
          )

  return conversation

In [ ]:
# @title

# construct prompt based on the task that user input
def test_get_prompt_based_on_task(query: str, text_context: str, tasks: list[str]) -> list[dict]:
  conversation = [
    {
      "role": "system",
      "content": [
        {"type": "text", "text": "You are a home cook chef and nutritionist assistant. Your primary function is to provide users with diet plans and meal preparation instructions, taking into account of the potential allergies the user may have."}
        ]
      }
    ]
  for task in tasks:
    if task in [CALORIES_CALCULATOR, COOK_HELPER, UNSPECIFIED, UNCLASSIFIED]:
      task_name = task
      break

  prompt_context = f"""Given the query: '{query}' and the following relevant snippets:

  {text_context}

  """

  if set([COOK_HELPER, CALORIES_CALCULATOR]).issubset(set(tasks)):
    prompt_context += f"""Recommend a recipe based on a given query and image using the provided data in the snippets. If there is not enough data in the snippets, you can use your knowledge. If the data is missing, you have to fill it in using your knowledge. The response must include rating, cooking time, ingredients, nutrition, and instructions. You also have to give a suggestion or opinion as a home cook chef for this food."""

  elif task_name == CALORIES_CALCULATOR:
    prompt_context += f"""Write the summary of the nutrition of this food based on the given query and image. If there is not enough data in the snippets, you can use your knowledge. If the data is missing, you have to fill it in using your knowledge. The response should includes ingredients, calories, fat, saturated fat, protein, cholesterol, sodium, fiber, sugar, protein. You also have to give a suggestion or opinion as a nutritionist for this food."""

  elif task_name == COOK_HELPER:
    prompt_context += f"""Recommend a recipe based on a given query and image using the provided data in the snippets. You have to be detailed with the ingredients and the instruction. If there is not enough data in the snippets, you can use your knowledge. If the data is missing, you have to fill it in using your knowledge. The response must include rating, cooking time, ingredients, and instructions. You also have to give a suggestion or opinion as a home cook chef for this food."""

  # TODO: test whether adding dietary filter prompt will improve the generation

  elif task_name == UNSPECIFIED:
    prompt_context += f"""Use the provided data in snippets and the image to discribe and conclude. You also have to give an opinion as a nutritionist and home cook chef for the image and data in the snippets"""

  elif task_name == UNCLASSIFIED:
    prompt_context += f"""Answer the question from the given query. You can use the data in the snippets as references. If there is not enough data in the snippets, you can use your knowledge. If the data is missing, you have to fill it in using your knowledge. You also have to give an opinion as a nutritionist and home cook chef to the query and image"""


  conversation.append(
        {
          "role": "user",
          "content": [
              {"type": "image"},
              {"type": "text", "text": prompt_context}
          ]
        }
        )

  return conversation

In [ ]:
# @title
def get_prompt(query: str, text_context: str, image_meta_data: dict[str, ...] ) -> list[dict]:
  conversation = [
    {
      "role": "system",
      "content": [
        {"type": "text", "text": "You are a home cook chef and nutritionist assistant. Your primary function is to provide users with diet plans and meal preparation instructions, taking into account potential allergies the user may have."}
        ]
    },
    {
        "role": "user",
        "content": [
            {"type": "image", "image": Image.open(image_meta_data[IMAGE_SAVE_PATH])},
            {"type": "text", "text": f"""
              Given the query "{query}" and the following relevant snippets:

              {text_context}
              and the image.

              Please provide a concise and accurate answer to the query, incorporating relevant information from the provided snippets where possible. If the data is missing or seem irrelevant fill the data with your own knowledge.
            """}
        ]
      }
    ]
  return conversation

In [ ]:
# @title
def test_get_prompt(query: str, text_context: str ) -> list[dict]:
  conversation = [
    # {
    #   "role": "system",
    #   "content": [
    #     {"type": "text", "text": "You are a home cook chef and nutritionist assistant. Your primary function is to provide users with diet plans and meal preparation instructions, taking into account potential allergies the user may have."}
    #     ]
    # },
    {
        "role": "user",
        "content": [
            {"type": "image", "image": Image.open("/content/Sandwich_2023.jpg")},
            {"type": "text", "text": f"""Given the query: '{query}' and the following relevant snippets:

              {text_context}

              Please provide a concise and accurate answer to the query, incorporating relevant information from the provided snippets where possible. If the data is missing or seem irrelevant fill the data with your own knowledge."""}
        ]
      }
    ]

  return conversation

### Test construct prompt

In [ ]:
text_content = peepo_process_retrieved_text(main_metadata_dict_list, indice_array, distance_array, [COOK_HELPER], '')

In [ ]:
task_prompt = test_get_prompt_based_on_task("Give me any sandwich recipe without milk", text_content, ["cook"])

In [ ]:
# @title
base_prompt = test_get_prompt("Give me any sandwich recipe without milk", text_content)

## 6. Pass the prompy through the LLM and get responses

In [ ]:
# @title
# OLD VERSION
import torch
from transformers import LlavaNextForConditionalGeneration, LlavaForConditionalGeneration, AutoProcessor, AutoModelForImageTextToText
from PIL import Image
from transformers import BitsAndBytesConfig
import numpy as np

# MODEL_NAME = "llava-hf/llava-v1.6-mistral-7b-hf"



# Configure 8-bit quantization
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.float16,
)

# Load the model with 8-bit quantization
processor = AutoProcessor.from_pretrained("llava-hf/llava-1.5-7b-hf")

model = LlavaForConditionalGeneration.from_pretrained(
    "llava-hf/llava-1.5-7b-hf",
    # torch_dtype=torch.float16,
    # low_cpu_mem_usage=True,
    quantization_config=bnb_config,
    device_map="auto"

)

# model = AutoModelForImageTextToText.from_pretrained("llava-hf/llava-v1.6-mistral-7b-hf", quantization_config=bnb_config, device_map="auto")


# model = LlavaNextForConditionalGeneration.from_pretrained(
#     MODEL_NAME,
#     torch_dtype=torch.float16,
#     low_cpu_mem_usage=True,
#     load_in_4bit=True
# )

# model = LlavaNextForConditionalGeneration.from_pretrained(
#     MODEL_NAME,
#     torch_dtype=torch.float16,
#     low_cpu_mem_usage=True,
#     use_flash_attention_2=True
# )





In [ ]:
# @title
from transformers import TextStreamer
conversation = [
    {
        "role": "user",
        "content": [
            {"type": "image", "image": Image.open("/content/Sandwich_2023.jpg")},
            {"type": "text", "text": "Can I eat this food, if I am allergic to milk?"},
        ],
    },
]

inputs = processor.apply_chat_template(
    task_prompt,
    add_generation_prompt=True,
    tokenize=True,
    return_dict=True,
    return_tensors="pt"
)
inputs = {k: v.to("cuda") for k,v in inputs.items()}

text_streamer = TextStreamer(processor, skip_prompt=True, skip_special_tokens=True)

# Generate
# print(text_streamer)
# streamer = model.generate(**inputs, max_new_tokens=4028, streamer=text_streamer)
streamer = model.generate(**inputs, streamer=text_streamer)
# output = model(**inputs, max_new_tokens=50)
# processor.batch_decode(output, skip_special_tokens=True)[0]


Based on the provided data, I recommend the following recipe:

**Recipe:**


In [ ]:
def generate_response_mistral(conversation: list[dict], image: Image.Image):
  prompt = llm_processor.apply_chat_template(conversation, add_generation_prompt=True)
  if image is None:
    inputs = llm_processor(prompt, return_tensors="pt").to(get_device())

  else:
    inputs = llm_processor(image, prompt, return_tensors="pt").to(get_device())
  text_streamer = TextStreamer(llm_processor, skip_prompt=True)
  # autoregressively complete prompt
  return llm_model.generate(**inputs, max_new_tokens=2880, streamer=text_streamer)

### Test llm model

In [ ]:
conversation = [
    {
        "role": "user",
        "content": [
            {"type": "image", "image": Image.open("/content/carbonara.jpg")},

            {"type": "text", "text": "Give me the recipe of the food in the image."},
        ],
    },
]
generate_response_mistral(conversation, None)

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


I'm sorry, but I am unable to provide a recipe for the food in the image as it is not clear what the food is. Can you please provide more information or a clearer image of the food? </s>


tensor([[    1,   733, 16289, 28793, 28705, 32000, 28705,    13, 28777,   495,
           528,   272, 13405,   302,   272,  2887,   297,   272,  3469, 28723,
           733, 28748, 16289, 28793,   315, 28742, 28719,  7371, 28725,   562,
           315,   837,  9638,   298,  3084,   264, 13405,   354,   272,  2887,
           297,   272,  3469,   390,   378,   349,   459,  3081,   767,   272,
          2887,   349, 28723,  2418,   368,  4665,  3084,   680,  1871,   442,
           264,  3081,   263,  3469,   302,   272,  2887, 28804, 28705,     2]],
       device='cuda:0')

# Test Full Flow

## User inputs text + image

In [ ]:
# @title
# get user input
user_text_input = "Give me the recipe of the sandwich without fish"
user_image_input = Image.open("/content/Sandwich_2023.jpg")


In [ ]:
# @title
# classify tasks
tasks = classify_tasks(user_text_input)

In [ ]:
# @title
# handle tasks
# check if tasks has allergen filter or food preference

# intention_query = user_text_input
# constraint = ""
# if ALLERGEN_FILTER in tasks or FOOD_PREFERENCE in tasks:
intention_query, constraint = separate_intent_and_constraint(user_text_input)

In [ ]:
# @title
# get embedded image
embedded_query_image = get_embedded_data(user_image_input, 'clip', 0 ,False)

In [ ]:
# @title
# retrieve food name from image
food_label_distance_array, food_name_indice_array = faiss_search(food_label_index, embedded_query_image, 1)

In [ ]:
# @title
# get food name
food_label_metadata = food_label_metadata_dict_list[food_name_indice_array[0]]

In [ ]:
# @title
intention_query += f" The food image label is {food_label_metadata[FOOD_LABEL]}."

In [ ]:
# @title
# get embedded query
embedded_query_text = get_embedded_data(intention_query, "sentence")

In [ ]:
# @title
# retrieve from RAG
distance_array, indice_array = faiss_search(text_index, embbed_query_text)

In [ ]:
# @title
# get text context
text_context = process_retrieved_text( metadata_dict_list, indice_array, distance_array, tasks, constraint)

In [ ]:
# @title
# construct conversation
conversation = get_prompt_based_on_task_mistral(intention_query, text_context , tasks)

In [ ]:
# @title
# generate response
generate_response_mistral(conversation, user_image_input)

## User inputs only text

## User inputs only image

## main function

In [ ]:
# @title
def talk_to_llm(query_text, query_image, debug: bool = False):
  if query_text is None and query_image is None:
    return "Please input either text or image"

  if query_text is not None:
    tasks = classify_tasks(query_text, True)

    print(f"{tasks=}")

    if UNCLASSIFIED not in tasks:
      intention_query, constraint = separate_intent_and_constraint(query_text)
      print(f"{intention_query=}")
      print(f"{constraint=}")

      if query_image is None:

        # clip_embedded_query_text = get_embedded_data(query_text, 'clip', 0, False)

        # distance_array, indice_array = faiss_search(food_image_index, clip_embedded_query_text)

        sentence_embedded_query_text = get_embedded_data(query_text, 'sentence', 0, False)

        distance_array, indice_array = faiss_search(food_label_sentence_index, sentence_embedded_query_text)

        if debug:
          print("text: yes, image: no = search image")
          print(food_image_metadata_dict_list[indice_array[0]])
        query_image = Image.open(food_image_metadata_dict_list[indice_array[0]][FOOD_IMAGE_PATH])

      else:
        clip_embedded_query_image = get_embedded_data(query_image, 'clip', 0, False)

        distance_array, indice_array = faiss_search(food_label_index, clip_embedded_query_image)


        food_label = food_label_metadata_dict_list[indice_array[0]][NAME]

        intention_query += f" The food image label is {food_label.replace('_', ' ')}."
        if debug:
          print("text: yes, image: yes = intent += food label")
          print(food_label_metadata_dict_list[indice_array[0]])
          print(intention_query)


  # image must not be None
  else:
    query_text = "Describe this food in the image."
    tasks = [UNSPECIFIED]
    constraint = ""

    clip_embedded_query_image = get_embedded_data(food_label_index, 'clip', 0 ,False)
    food_label_distance_array, food_label_indice_array = faiss_search(food_label_index, embedded_query_image, 1)
    if debug:
      print("text: no, image: yes = search food label")
      print(food_label_metadata_dict_list[indice_array[0]])
    food_label = food_label_metadata_dict_list[food_label_indice_array[0]][NAME]
    intention_query = f"food name: {food_label}."

  if UNCLASSIFIED not in tasks:
    sentence_embedded_query_image = get_embedded_data(intention_query, 'sentence', 0, False)

    distance_array, indice_array = faiss_search(main_index, sentence_embedded_query_image)

    text_content = process_retrieved_text(main_metadata_dict_list, indice_array, distance_array, tasks, constraint)
  else:
    text_content = ""

  if debug:
    print("text_content")
    print(text_content)

  conversation = get_prompt_based_on_task_mistral(query_text, text_content, tasks)

  return generate_response_mistral(conversation, original_image)







In [ ]:
def talk_to_llm(query_text, query_image, conversation: list[dict], debug: bool = False):
  if query_text is None and query_image is None:
    return "Please input either text or image"

  if query_text is not None:
    tasks = classify_tasks(query_text, debug)

    intention_query, constraint = separate_intent_and_constraint(query_text)

    if debug:
      print(f"{intention_query=}")
      print(f"{constraint=}")

    if query_image is None:

      clip_embedded_query_image = get_embedded_data(query_image, 'clip', 0, False)

      distance_array, indice_array = faiss_search(food_label_index, clip_embedded_query_image)


      food_label = food_label_metadata_dict_list[indice_array[0]][NAME]

      intention_query += f" The food is {food_label.replace('_', ' ')}."
      if debug:
        print("text: yes, image: yes = intent += food label")
        print(food_label_metadata_dict_list[indice_array[0]])
        print(intention_query)


  # image must not be None
  else:
    query_text = "Describe this food in the image."
    tasks = [UNSPECIFIED]
    constraint = ""

    clip_embedded_query_image = get_embedded_data(food_label_index, 'clip', 0 ,False)
    food_label_distance_array, food_label_indice_array = faiss_search(food_label_index, embedded_query_image, 1)

    food_label = food_label_metadata_dict_list[food_label_indice_array[0]][NAME]
    intention_query = f" The food is {food_label}."

    if debug:
      print("text: no, image: yes = search food label")
      print(food_label_metadata_dict_list[indice_array[0]])
      print(intention_query)

  sentence_embedded_query_text = get_embedded_data(intention_query, 'sentence', 0, False)

  text_content = reiterate_discarded(sentence_embedded_query_text, tasks, constraint)

  if debug:
    print("text_content")
    print(text_content)

  conversation = get_prompt_based_on_task_mistral(conversation, query_text, query_image, text_content, tasks)

  return generate_response_mistral(conversation, query_image)

In [ ]:
# @title
def talk_to_llm_copy(query_text, query_image, debug: bool = False):
  if query_text is None and query_image is None:
    return "Please input either text or image"

  if query_text is not None:
    tasks = classify_tasks(query_text, debug)

    intention_query, constraint = separate_intent_and_constraint(query_text)

    if debug:
      print(f"{intention_query=}")
      print(f"{constraint=}")

    if query_image is None:

      # clip_embedded_query_text = get_embedded_data(query_text, 'clip', 0, False)

      # distance_array, indice_array = faiss_search(food_image_index, clip_embedded_query_text)

      # sentence_embedded_query_text = get_embedded_data(query_text, 'sentence', 0, False)

      # distance_array, indice_array = faiss_search(food_label_sentence_index, sentence_embedded_query_text)

      # if debug:
      #   print("text: yes, image: no = search image")
      #   print(food_image_metadata_dict_list[indice_array[0]])
      # query_image = Image.open(food_image_metadata_dict_list[indice_array[0]][FOOD_IMAGE_PATH])
      pass

    else:
      clip_embedded_query_image = get_embedded_data(query_image, 'clip', 0, False)

      distance_array, indice_array = faiss_search(food_label_index, clip_embedded_query_image)


      food_label = food_label_metadata_dict_list[indice_array[0]][NAME]

      intention_query += f" The food is {food_label.replace('_', ' ')}."
      if debug:
        print("text: yes, image: yes = intent += food label")
        print(food_label_metadata_dict_list[indice_array[0]])
        print(intention_query)


  # image must not be None
  else:
    query_text = "Describe this food in the image."
    tasks = [UNSPECIFIED]
    constraint = ""

    clip_embedded_query_image = get_embedded_data(food_label_index, 'clip', 0 ,False)
    food_label_distance_array, food_label_indice_array = faiss_search(food_label_index, embedded_query_image, 1)

    food_label = food_label_metadata_dict_list[food_label_indice_array[0]][NAME]
    intention_query = f" The food is {food_label}."

    if debug:
      print("text: no, image: yes = search food label")
      print(food_label_metadata_dict_list[indice_array[0]])
      print(intention_query)

  sentence_embedded_query_image = get_embedded_data(intention_query, 'sentence', 0, False)

  distance_array, indice_array = faiss_search(main_index, sentence_embedded_query_image)

  text_content = process_retrieved_text(main_metadata_dict_list, indice_array, distance_array, tasks, constraint)

  if debug:
    print("text_content")
    print(text_content)

  conversation = get_prompt_based_on_task_mistral(query_text, text_content, tasks)

  return generate_response_mistral(conversation, query_image)







In [ ]:
# test

response = talk_to_llm("Give me the recipe of the food in the image.", Image.open('/content/carbonara.jpg'), False)

image


100%|██████████| 1/1 [00:00<00:00, 21.65it/s]


text


100%|██████████| 1/1 [00:00<00:00, 73.26it/s]
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


current non discarded count: 5
Spaghetti Alla Carbonara Recipe

Ingredients:

* 1 pound spaghetti
* 1/2 cup extra virgin olive oil
* 4 ounces pancetta, diced
* 4 ounces thick slab bacon, diced
* 4 garlic cloves, minced
* 4 large eggs
* 1 cup grated Parmesan cheese
* Freshly ground black pepper, to taste
* Fresh flat-leaf parsley, chopped

Instructions:

1. Bring a large pot of salted water to a boil. Add the spaghetti and cook for 8 to 10 minutes or until tender yet firm. Drain the pasta well, reserving 1/2 cup of the starchy cooking water to use in the sauce if you wish.

2. Meanwhile, heat the olive oil in a deep skillet over medium flame. Add the pancetta and sauté for about 3 minutes, until the bacon is crisp and the fat is rendered. Toss the garlic into the fat and sauté for less than 1 minute to soften.

3. Add the hot, drained spaghetti to the pan and toss for 2 minutes to coat the strands in the bacon fat.

4. Beat the eggs and Parmesan together in a mixing bowl, stirring well 

In [ ]:
def talk_to_llm2(a,b):
  if a is not None and b is not None:
    yield "both "
    for word in _LOREM_IPSUM.split(" "):
      yield word + " "
      time.sleep(0.02)
  else:
    yield "some "
    for word in _LOREM_IPSUM.split(" "):
        yield word + " "
        time.sleep(0.02)


# Create app and run

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.7/2.7 MB 36.9 MB/s eta 0:00:00


In [ ]:
# mount drive and kaggle hub
from google.colab import drive
import kagglehub
drive.mount("/content/drive")
KAGGLE_PATH = kagglehub.dataset_download("kmader/food41")

Mounted at /content/drive


In [ ]:

%%writefile app.py
# load all dependencies
from threading import Thread
from streamlit_tags import st_tags, st_tags_sidebar
import streamlit as st
from transformers import CLIPModel, CLIPProcessor, pipeline, TextIteratorStreamer, BitsAndBytesConfig, LlavaNextForConditionalGeneration, AutoProcessor
from sentence_transformers import SentenceTransformer, CrossEncoder

from PIL import Image
from IPython.display import display, HTML

import faiss
import numpy as np
import pandas as pd

import torch

import requests
from pathlib import Path
import os
import sys
import json
import math
import re

import time
from tqdm import tqdm
from tqdm import notebook
from copy import deepcopy

print("done import")

# Define constants
FOOD_NAME = "food name"
FOOD_IMAGE = "food image"
FOOD_IMAGE_PATH = "food image path"
FOOD_INGREDIENT = "food ingredient"
FOOD_INSTRUCTION = "food instruction"

# Dataset
NAME = "name"
COOK_TIME = "cook_time"
PREP_TIME = "prep_time"
DESCRIPTION = "description"
RECIPE_CATEGORY = "recipe_category"
INGREDIENT_QUANTITIES = "ingredient_quantites"
INGREDIENT_PARTS = "ingredient_parts"
RATING = "rating"
CALORIES = "calories"
FAT = "fat"
SATURATED_FAT = "saturated_fat"
CHOLESTEROL = "cholesterol"
SODIUM = "sodium"
CARBOHYDRATE = "carbohydrate"
FIBER = "fiber"
SUGAR = "sugar"
PROTEIN = "protein"
RECIPE_SERVING = "recipe_serving"
RECIPE_YIELD = "recipe_yield"
RECIPE_INSTRUCTION = "recipe_instruction"
ALLERGY = "allergy"

# Task
CALORIES_CALCULATOR = "nutrition"
COOK_HELPER = "recipe provider"
ALLERGEN_FILTER = "allergen avoidance"
FOOD_PREFERENCE = "food tag"
MEAL_PLANNER = "diet meal planner"

LEFTOVER_REMIXER = "recipe from ingredient"
DIETARY_FILTER = "dietary filter"
UNSPECIFIED = "unspecified"
UNCLASSIFIED = "unclassified"

TASK_LIST = [CALORIES_CALCULATOR, ALLERGEN_FILTER, FOOD_PREFERENCE, COOK_HELPER, MEAL_PLANNER]

# Model
TASK_CLASSIFIER_MODEL_NAME = "facebook/bart-large-mnli"
TASK_CLASSIFIER_THRESHOLD = 0.65
RAG_DISTANCE_THRESGHOLD = 0.65

CLIPMODEL_NAME = "openai/clip-vit-base-patch32"

SENTENCE_MINI_MODEL = "all-MiniLM-L6-v2"
SENTENCE_BEST_MODEL = "all-mpnet-base-v2"



# Path
SAVE_IMAGE_PATH = Path("/content/images")
SAVE_EMBEDDED_DATA_PATH = Path("/content/embedding")
SAVE_INDEX_PATH = Path("/content/faiss_index")
FOOD_DATABASE_PATH = Path("/content/drive/MyDrive/nlp_database/food_database.json")
NLP_DATABASE = Path("/content/drive/MyDrive/nlp_database")
KAGGLE_PATH = Path("/kaggle/input/food41")

# Faiss
K_ORDER = 5
start = time.time()
def get_device() -> str:
  if torch.cuda.is_available():
    return "cuda"
  elif torch.backends.mps.is_available():
    return "mps"
  else:
    return "cpu"

def load_embedded_data(file_name: str) -> np.ndarray:
  return np.load(NLP_DATABASE / f"{file_name}.npy" )

def load_faiss_index(index_name: str):
  return faiss.read_index(str(NLP_DATABASE / f"{index_name}.index"))



# classify task
def classify_tasks(query_text: str, debug: bool = False) -> list[str]:
  result = task_classifier_pipeline(query_text, TASK_LIST, multi_label=True)
  if debug:
    print(result)
  tasks = []
  for i, score in enumerate(result["scores"]):
    if score > TASK_CLASSIFIER_THRESHOLD:
      tasks.append(result["labels"][i])

  # handle the unknown task
  if not tasks or (COOK_HELPER not in tasks and CALORIES_CALCULATOR not in tasks and MEAL_PLANNER not in tasks):
    tasks.append(UNCLASSIFIED)

  return tasks

# separate intention and constraint
def separate_intent_and_constraint(user_text: str) -> tuple[str, list[str]]:
    """
    Function to separate constriant of the query from the intention query.
    This function should be used to only the query that contains Dietary Filter Task
    """

    constraint_keywords = ["but", "without", "avoid", "except", "no", "must be", "should be"]

    lower_text = user_text.lower()

    for keyword in constraint_keywords:
        if keyword in lower_text:
            parts = re.split(rf"\b{keyword}\b", user_text, maxsplit=1, flags=re.IGNORECASE)
            intent_query = parts[0].strip()
            constraint = parts[1].strip() if len(parts) > 1 else ""

            intent_query = intent_query.rstrip(",.?!").rstrip()
            intent_query = re.sub(r'\b(with|without|but|except|avoid|no|must be|should be)\b$', '', intent_query, flags=re.IGNORECASE).strip()

            constraint = constraint.strip('.,?')
            return intent_query, [constraint]

    return user_text.strip(), []

# create metadata and rich text from merged datasets
def get_metadata_dict_list(json_database_path: Path) -> list[dict[str, list[str] | str]]:
  if not os.path.exists(json_database_path):
    raise FileNotFoundError(f"File not found: {json_database_path}")

  with open(json_database_path, "r", encoding="utf-8") as f:
    metadata_dict_list = json.load(f)

  def process_column(metadata_dict: dict[str,str]):
    for key, value in metadata_dict.items():
      if value is None:
        metadata_dict[key] = "missing" # TODO: Come up with a better text to let llm know that they have to search if it encounters this text
      elif key == ALLERGY:
        metadata_dict[key] = value.split(", ")
      elif key == INGREDIENT_QUANTITIES:
        try:
          metadata_dict[key] = eval(value.replace("NA", '"missing"'))
        except:
          metadata_dict[key] = "missing"
      elif key in [INGREDIENT_PARTS, RECIPE_INSTRUCTION]:
        try:
          metadata_dict[key] = eval(value)
        except:
          metadata_dict[key] = "missing"
    return metadata_dict

  # to process the str list to list and fill the missing value for llm to search
  metadata_dict_list = list(map(process_column, metadata_dict_list))

  return metadata_dict_list

# create rich text
def create_rich_text(metadata_dict_list: dict[str, list[str] | str]) -> list[str]:
  rich_text_list = []

  # construct rich text to store in the vector database
  for metadata_dict in tqdm(metadata_dict_list):
    if metadata_dict is None:
      continue
    rich_text_list.append(
        f"Recipe: {metadata_dict[NAME]}. Dish type: {metadata_dict[RECIPE_CATEGORY]}. Main ingredients: {', '.join(metadata_dict[INGREDIENT_PARTS])}, Serving: {metadata_dict[RECIPE_SERVING]} people. Review rating: {metadata_dict[RATING]}. Cook time: {metadata_dict[COOK_TIME]}. Preparation time: {metadata_dict[PREP_TIME]}. Nutrition - Calories: {metadata_dict[CALORIES]}, Fat: {metadata_dict[FAT]}, Sodium: {metadata_dict[SODIUM]}, Cholesterol: {metadata_dict[CHOLESTEROL]}, Sugar: {metadata_dict[SUGAR]}, Protein: {metadata_dict[PROTEIN]}."
        )

  return rich_text_list

# save image and create image metadata
def create_food_image_metadata_dict_list(path) -> list[str]:
  food_image_metadata_dict_list = []
  path_test = KAGGLE_PATH / "meta/meta/test.txt"
  if os.path.exists(path_test):
    with open(path_test, 'r') as file:
        content = file.read()
  # construct rich image to store in the vector database
  for sub in tqdm(content.split('\n')):
    if sub is None:
      continue
    elif sub == "":
      continue
    path_image = KAGGLE_PATH / f"images/{sub}.jpg"
    if os.path.exists(path_image):
      food_image_metadata_dict_list.append({
          NAME: sub.split('/')[0],
          FOOD_IMAGE_PATH: path_image
      })
    else:
      print(sub)

  return food_image_metadata_dict_list



# embedding rich text and image

batch_size = 16
batch_save = 250

def get_embedded_data(data_list: list[str | Image.Image] | str | Image.Image, embedding_model: str, resume_index: int = 0, save: bool = True) -> torch.Tensor:
  embedded_data_list = []

  if not isinstance(data_list, list):
    data_list = [data_list]

  data_type = "image" if isinstance(data_list[0], Image.Image) else "text"
  print(data_type)

  for i in tqdm(range(resume_index, math.ceil(len(data_list) / batch_size))):
    spliced_data_list = data_list[i * batch_size : min(len(data_list), (i + 1) * batch_size)]
    if embedding_model == "clip":
      if data_type == "image":
        processed_data_inputs = clip_processor(images=spliced_data_list, return_tensors="pt", padding=True, truncation=True)

        processed_data_inputs = {k : v.to(get_device()) for k,v in processed_data_inputs.items()}

        embedded_data_list.extend(clip_model.get_image_features(**processed_data_inputs).detach().cpu().numpy())
      elif data_type == "text":
        processed_data_inputs = clip_processor(spliced_data_list, return_tensors="pt", padding=True, truncation=True)

        processed_data_inputs = {k : v.to(get_device()) for k,v in processed_data_inputs.items()}
        embedded_data_list.extend(clip_model.get_text_features(**processed_data_inputs).detach().cpu().numpy())

    elif embedding_model == "sentence":
      embedded_data_list.extend(sentences_model.encode(spliced_data_list))


  embedded_data_array = np.array(embedded_data_list)

  return embedded_data_array

# Create index using faiss
def get_faiss_index(embedded_data: np.ndarray, method: str = "flat", save: bool = False, name: str = "") -> faiss.IndexFlatL2:
  index = faiss.IndexFlatL2(embedded_data.shape[1]) # change to HNWS
  index.add(embedded_data)

  return index

# Faiss similarity search
def faiss_search(index: faiss.IndexFlatL2, embbeded_query: np.ndarray, k: int = K_ORDER) -> tuple[np.ndarray, np.ndarray]:
  distance_array, indice_array = index.search(embbeded_query, k)
  return distance_array[0], indice_array[0]



# process retrieved text after retrieving from rag
def process_retrieved_text(
                           indice_array: np.ndarray, distance_array: np.ndarray,
                           tasks: list[str],
                           constraint_list:list[str]) -> tuple[list[str], bool]: # implement the constraint to be accepted as list

  text_content_list = []
  should_end = False
  is_filtering_allergy = ALLERGEN_FILTER in tasks or FOOD_PREFERENCE in tasks

  for i, indice in enumerate(indice_array):
      main_metadata_dict = main_metadata_dict_list[indice]

      if distance_array[i] > RAG_DISTANCE_THRESGHOLD:
        should_end = True
        break

      if is_filtering_allergy or constraint_list:
        allergy = False
        for constraint in constraint_list:
          if (
              str(main_metadata_dict[INGREDIENT_PARTS]).lower().find(constraint.lower()) != -1 or
              str(main_metadata_dict[ALLERGY]).lower().find(constraint.lower()) != -1 or
              str(main_metadata_dict[RECIPE_INSTRUCTION]).lower().find(constraint.lower()) != -1
              ):
            allergy = True
            break

        if allergy:
            continue


      text_content_list.append(f"""[**Food Name**: {main_metadata_dict[NAME]}
**User Rating**: {main_metadata_dict[RATING]} stars
**Food Category**: {main_metadata_dict[RECIPE_CATEGORY]}
**Cook Time**: {main_metadata_dict[COOK_TIME]}
**Preparation Time**: {main_metadata_dict[PREP_TIME]}
**Ingredient Parts**: {", ".join(main_metadata_dict[INGREDIENT_PARTS])}
**Ingredient Quantities**: {", ".join(main_metadata_dict[INGREDIENT_QUANTITIES])}
**Nutrition**:
- Calories: {main_metadata_dict[CALORIES]} cals
- Carbohydrate: {main_metadata_dict[CARBOHYDRATE]} g
- Protein: {main_metadata_dict[PROTEIN]} g
- Fat: {main_metadata_dict[FAT]} g
- Saturated Fat: {main_metadata_dict[SATURATED_FAT]} g
- Fiber: {main_metadata_dict[FIBER]} g
- Cholesterol: {main_metadata_dict[CHOLESTEROL]} mg
- Sodium: {main_metadata_dict[SODIUM]} mg
- Sugar: {main_metadata_dict[SUGAR]} g
**Serving**: {main_metadata_dict[RECIPE_SERVING]} people
**Recipe Instruction**: {", ".join(main_metadata_dict[RECIPE_INSTRUCTION])}
**Not Suitable for People Who has Allergy**: {", ".join(main_metadata_dict[ALLERGY])}]
\n""")


  return text_content_list, should_end

# reiterate to filter out the constraints
def reiterate_discarded(embedded_query_text: str,
                        tasks: list[str],
                        constraint_list:list[str],
                        debug: bool = False) -> str:

  should_end = False
  current_discarded_count = 0
  current_non_discarded_count = 0

  while True:
    distance_array, indice_array = faiss_search(main_index, embedded_query_text, k= K_ORDER + current_discarded_count)

    text_content, should_end = process_retrieved_text(indice_array, distance_array, tasks, constraint_list)

    current_non_discarded_count = len(text_content)
    current_discarded_count += K_ORDER - len(text_content)

    if not (current_non_discarded_count < K_ORDER and not should_end):
      break

    if debug:
      print(f"current non discarded count: {current_non_discarded_count}")
      print(f"current discarded count: {current_discarded_count}")

  if not text_content:
    text_content = ["There is no context. Please use your knowledge."]
  if debug:
    print(f"text content: {text_content}")

  return text_content

def rerank_text_content(text_content, query_text):

    scores = reranking_model.predict([(query_text, text) for text in text_content])

    # Sort by score in descending order
    ranked_results = sorted(zip(text_content, scores), key=lambda x: x[1], reverse=True)

    sorted_text_content = [text_content for text_content, score in ranked_results]

    return sorted_text_content

def get_prompt_based_on_task_mistral(conversation: list[dict], query: str, image: Image, text_content: str, tasks: list[str]) -> list[dict]:
    #, taking into account of the potential allergies the user may have.
  for task in tasks:
    if task in [CALORIES_CALCULATOR, COOK_HELPER, MEAL_PLANNER, UNSPECIFIED, UNCLASSIFIED]:
      task_name = task
      break

  prompt_context = f"""Given the query: '{query}' and the following relevant snippets:

  {text_content}

  """

  # if set([COOK_HELPER, CALORIES_CALCULATOR]).issubset(set(tasks)):
  #   prompt_context += f"""Recommend a recipe based on a given query and image using the provided data in the snippets. If there is not enough data in the snippets, you can use your knowledge. If the data is missing, you have to fill it in using your knowledge. The response must include rating, cooking time, ingredients, nutrition, and instructions. You also have to give a suggestion or opinion as a home cook chef for this food."""
  if task_name == MEAL_PLANNER:
    prompt_context += """Answer the given query. You can use both your knowledge and data in the snippet. If the some information is 'missing' you can use your knowledge to fill in that question or leave out those information. The response should be in a readable format for a meal planner. Then ask user what else do you want to know."""

  elif task_name == CALORIES_CALCULATOR:
    prompt_context += """Answer the given query. You can use both your knowledge and data in the snippet. If the some information is 'missing' you can use your knowledge to fill in that question or leave out those information. The response should be in a readable format for nutrition summary. Then ask user what else do you want to know."""
    # """Write the summary of the nutrition of this food based on the given query and image. If there is not enough data in the snippets, you can use your knowledge. If the data is missing, you have to fill it in using your knowledge. The response should includes ingredients, calories, fat, saturated fat, protein, cholesterol, sodium, fiber, sugar, protein. You also have to give a suggestion or opinion as a nutritionist for this food."""

  elif task_name == COOK_HELPER:
    prompt_context += """Answer the given query. You can use both your knowledge and data in the snippets. If the some information is 'missing' you can use your knowledge to fill in that question or leave out those information. The response should be in a readable format for recipe and home cook. Then ask user what else do you want to know."""
    # """Recommend a recipe based on a given query and image using the provided data in the snippets. You have to be detailed with the ingredients and the instruction. If there is not enough data in the snippets, you can use your knowledge. If the data is missing, you have to fill it in using your knowledge. The response must include rating, cooking time, ingredients, and instructions. You also have to give a suggestion or opinion as a home cook chef for this food."""

  # TODO: test whether adding dietary filter prompt will improve the generation

  elif task_name == UNSPECIFIED:
    prompt_context += f"""Use the provided data in snippets and the image to summary and conclude. You also have to give an opinion as a nutritionist and home cook chef for the image and data in the snippets. Then ask user what else do you want to know."""

  elif task_name == UNCLASSIFIED:
    prompt_context += f"""Answer the given query using your own knowledge or provided data in the snippets. The response should be in a readable format. Then ask user what else do you want to know."""
    #  You can use the data in the snippets as references. If there is not enough data in the snippets, you can use your knowledge. If the data is missing, you have to fill it in using your knowledge. You also have to give an opinion as a nutritionist and home cook chef to the query and image"""

  if image is not None:
    conversation.append(
          {
            "role": "user",
            "content": [
                {"type": "image"},
                {"type": "text", "text": prompt_context}
            ]
          }
          )
  else:
    conversation.append(
          {
            "role": "user",
            "content": [
                {"type": "text", "text": prompt_context}
            ]
          }
          )

  return conversation

def generate_response_mistral(conversation: list[dict], image: list[Image.Image]):
  prompt = llm_processor.apply_chat_template(conversation, add_generation_prompt=True)
  if not image:
    inputs = llm_processor(prompt, return_tensors="pt").to(get_device())

  else:
    inputs = llm_processor(image, prompt, return_tensors="pt").to(get_device())

  text_streamer = TextIteratorStreamer(llm_processor, skip_prompt=True)

  # Run the generation in a separate thread, so that we can fetch the generated text in a non-blocking way.

  generation_kwargs = dict(inputs, streamer=text_streamer, max_new_tokens=2880)

  thread = Thread(target=llm_model.generate, kwargs=generation_kwargs)

  thread.start()

  return text_streamer


def talk_to_llm(query_text, query_image, conversation: list[dict], image_list: list[Image.Image], constraint_list: list[str], debug: bool = False):
  constraint_list = [] if constraint_list is None else constraint_list
  if query_text == "" and query_image is None:
    return "Please input either text or image"

  if query_text != "":
    tasks = classify_tasks(query_text, debug)

    intention_query, constraint = separate_intent_and_constraint(query_text)

    if constraint_list:
      query_text += f" Avoid / without {', '.join(constraint_list)}."
    constraint_list += constraint
    if debug:
      print(f"{intention_query=}")
      print(f"{constraint_list=}")

    if query_image is not None:
      image_list.append(query_image)
      st.session_state.image_list = image_list

      clip_embedded_query_image = get_embedded_data(query_image, 'clip', 0, False)

      distance_array, indice_array = faiss_search(food_label_index, clip_embedded_query_image)


      food_label = food_label_metadata_dict_list[indice_array[0]][NAME]

      if intention_query[-1] not in ["!", ".", "?"]:
        intention_query += "."
      intention_query += f" The food is {food_label.replace('_', ' ')}."
      if debug:
        print("text: yes, image: yes = intent += food label")
        print(food_label_metadata_dict_list[indice_array[0]])
        print(intention_query)


  # image must not be None
  else:
    query_text = "Describe this food in the image."
    tasks = [UNSPECIFIED]
    constraint = []

    clip_embedded_query_image = get_embedded_data(query_image, 'clip', 0 ,False)
    food_label_distance_array, food_label_indice_array = faiss_search(food_label_index, clip_embedded_query_image, 1)

    food_label = food_label_metadata_dict_list[food_label_indice_array[0]][NAME]
    intention_query = f" The food is {food_label}."

    if debug:
      print("text: no, image: yes = search food label")
      print(food_label_metadata_dict_list[indice_array[0]])
      print(intention_query)

  sentence_embedded_query_text = get_embedded_data(intention_query, 'sentence', 0, False)
  text_content = reiterate_discarded(sentence_embedded_query_text, tasks, constraint_list, debug)
  if debug:
    print(f"len text_content: {len(text_content)}")

  sorted_text_content = rerank_text_content(text_content, query_text)

  if debug:
    print("before")
    print(text_content)
    print("after")
    print(sorted_text_content)

  joined_text_content = "".join(sorted_text_content)

  conversation = get_prompt_based_on_task_mistral(conversation, query_text, query_image, joined_text_content, tasks)
  st.session_state.conversation = conversation
  if debug:

    print("conversation")
    print(conversation)
    print("image_list")
    print(image_list)

  return generate_response_mistral(conversation, image_list)


def init_page() -> None:
  st.set_page_config(
    page_title="Food Assistance Chatbot"
  )
  st.header("Food Assistance Chatbot")
  st.sidebar.title("Options")

def init_messages() -> None:
  clear_button = st.sidebar.button("Clear Conversation", key="clear")
  if 'processing' not in st.session_state:
    st.session_state.processing = False

  if clear_button or "messages" not in st.session_state:
    st.session_state.messages = []
    st.session_state.conversation = [
    {
      "role": "system",
      "content": [
        {"type": "text", "text": "You are a home cook chef and nutritionist assistant. Your primary function is to provide users with diet plans, nutrition, recipe, and meal preparation instructions."}
        ]
      }]
    st.session_state.image_list = []


@st.cache_resource
def load_all_model():
  task_classifier_pipeline = pipeline("zero-shot-classification", model=TASK_CLASSIFIER_MODEL_NAME)

  clip_model = CLIPModel.from_pretrained(CLIPMODEL_NAME).to(get_device())
  clip_processor = CLIPProcessor.from_pretrained(CLIPMODEL_NAME)

  sentences_model = SentenceTransformer(SENTENCE_MINI_MODEL).to(get_device())

  reranking_model = CrossEncoder('cross-encoder/ms-marco-MiniLM-L-6-v2')

  bnb_config = BitsAndBytesConfig(
      load_in_4bit=True,
      bnb_4bit_quant_type="nf4",
      bnb_4bit_compute_dtype=torch.float16,
  )

  llm_model = LlavaNextForConditionalGeneration.from_pretrained(
      "llava-hf/llava-v1.6-mistral-7b-hf",
      quantization_config=bnb_config,
      device_map="auto"
  )
  llm_processor = AutoProcessor.from_pretrained("llava-hf/llava-v1.6-mistral-7b-hf")

  main_metadata_dict_list = get_metadata_dict_list(FOOD_DATABASE_PATH)
  food_image_metadata_dict_list = create_food_image_metadata_dict_list(KAGGLE_PATH)
  food_label_metadata_dict_list = food_image_metadata_dict_list

  main_index = load_faiss_index("main_index_1745901394.7850068")
  food_label_index = load_faiss_index("food_label_clip_index_1745943408.8384023")
  with open(NLP_DATABASE / "food_allergens.json", "r", encoding="utf-8") as f:
      food_allergen_table = json.load(f)

  food_allergy_list = list(set([metadata["allergy"].strip() for metadata in food_allergen_table] + [metadata["name"].strip() for metadata in food_allergen_table]))
  return task_classifier_pipeline, clip_model, clip_processor, sentences_model, reranking_model, llm_model, llm_processor, main_metadata_dict_list, food_image_metadata_dict_list, food_label_metadata_dict_list, main_index, food_label_index, food_allergy_list

init_page()

start = time.time()
task_classifier_pipeline, clip_model, clip_processor, sentences_model, reranking_model, llm_model, llm_processor,main_metadata_dict_list, food_image_metadata_dict_list, food_label_metadata_dict_list, main_index, food_label_index, food_allergy_list = load_all_model()
print(f"finished load {time.time() - start}")

def create_message():
  messages = st.session_state.get("messages", [])
  for message in messages:
    with st.chat_message(message["role"]):
      if isinstance(message["content"], Image.Image):
        st.image(message["content"])
      else:
        st.markdown(message["content"])

def get_conversation_and_image():
  conversation = st.session_state.get("conversation", [
    {
      "role": "system",
      "content": [
        {"type": "text", "text": "You are a home cook chef and nutritionist assistant. Your primary function is to provide users with diet plans, nutrition, recipe, and meal preparation instructions."}
        ]
      }])
  image_list = st.session_state.get("image_list", [])

  return conversation, image_list



# Function to set processing state to true
def start_processing():
    st.session_state.processing = True

# Function to set processing state to false
def end_processing():
    st.session_state.processing = False

def main():
    init_messages()
    create_message()

    user_input = st.text_area("Input your question!", height=68, disabled=st.session_state.processing)
    image_input = st.file_uploader("Upload an image (optional)", type=["jpg", "jpeg", "png"], disabled=st.session_state.processing)
    constraint_list = st_tags(label = "Enter your allergy or food you want to avoid",
                  suggestions = food_allergy_list,
                   text="Please enter to add more",
                   maxtags = 4)

    if st.button("Submit", disabled=st.session_state.processing):

      start_processing()

      if user_input == "" and image_input is None:
        st.error("Please input either text or image",icon="🚨")
      else:
        image = Image.open(image_input) if image_input is not None else image_input
        if image is not None:
          st.session_state.messages.append({"role": "user", "content": image})
        if user_input != "":
          st.session_state.messages.append({"role": "user", "content": user_input})
        with st.chat_message("assistant"):
          conversation, image_list = get_conversation_and_image()
          streamer = talk_to_llm(user_input, image, conversation, image_list, constraint_list, True)
          placeholder = st.empty()
          full_response = ""
          for chunk in streamer:
            full_response += chunk
            placeholder.markdown(full_response)
          placeholder = st.empty()
          conversation.append({
          "role": "assistant",
          "content": [
              {"type": "text", "text": full_response}
              ]
            })
          st.session_state.conversation = conversation
        st.session_state.messages.append({"role": "assistant", "content": full_response})

      end_processing()


if __name__ == "__main__":
    main()



Overwriting app.py


Overwriting app.py


In [ ]:
import torch
torch.cuda.empty_cache()


In [ ]:
!curl https://loca.lt/mytunnelpassword

34.125.142.190

In [ ]:
!streamlit run app.py & npx localtunnel --port 8501



⠙⠹⠸⠼⠴
  You can now view your Streamlit app in your browser.

  Local URL: http://localhost:8501
  Network URL: http://172.28.0.12:8501
  External URL: http://34.125.142.190:8501

⠦your url is: https://icy-baths-cheer.loca.lt
2025-05-03 17:47:37.084635: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1746294457.117240   17135 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1746294457.127135   17135 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-05-03 17:47:37.166543: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the followin